In [12]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets 

import time
import discretize

from SimPEG import (
    maps,
    utils,
    survey,
    data,
    data_misfit,
    directives,
    optimization,
    regularization,
    inverse_problem,
    inversion,
)